In [1]:
import json
import requests
import warnings
from typing import Optional
import os
import time
import pandas as pd
from tqdm import tqdm
import shutil

In [6]:
def pdb_to_pdbnew(pdb_id='7RFS'):
    with open(f'./gett_pdb/{pdb_id}/{pdb_id}_mol2/{pdb_id}.pdb') as f:
        line = f.readlines()
    true_len = len(line)
    for i in range(true_len):
        if line[i][0:4] == 'ATOM':
            del line[0:i]
            break
    for i in range(len(line)):
        if line[i][0:6] == 'CONECT':
            del line[i:]
            break
    line_w = []
    for i in range(len(line)):
        if line[i][0:4] == 'HETA':
            chain_name = line[i+5][21]
            break      
    for i in range(len(line)):
        if line[i][21] == chain_name:
            line_w.append(line[i])
    file_new = open(f'./gett_pdb/{pdb_id}/{pdb_id}_mol2/{pdb_id}_new.pdb', 'w')
    file_new.writelines(line_w)
    file_new.close()
    os.mkdir(f'./gett_pdb/{pdb_id}/{pdb_id}_mol2/ligand/')
    # 查找配体HETA所在的链并且只保留这条链对应的坐标

In [45]:
def pdb_to_ligandpdb(pdb_id='7RFS',ligand_name = '4WI'):
    line_w = []
    with open(f'./gett_pdb/{pdb_id}/{pdb_id}_mol2/{pdb_id}_new.pdb') as f:
        line = f.readlines()
    true_len = len(line)
    for i in range(true_len):
        if line[i][0:4] == 'HETA':
            ligand_num = line[i][23:27]
            break
    for i in range(true_len):
        if line[i][0:4] == 'HETA':
            if line[i][17:20] == ligand_name:
                if line[i][23:27] == ligand_num:
                    line_w.append(line[i])
    if len(line_w) > 15:
        file_new = open(f'./gett_pdb/{pdb_id}/{pdb_id}_mol2/ligand/{pdb_id}_{ligand_name}.pdb', 'w')
        file_new.writelines(line_w)
        file_new.close()
    # 增加了对小分子配体()的判断,如果小分子配体的原子数大于15个,则认为是正确的配体,否则不是正确的配体
    # 对于小分子配体,只保留一个配体的坐标


In [43]:
pdb_id = '5SL8'
os.path.getsize(f'./gett_pdb/{pdb_id}/{pdb_id}_mol2/')

4096

In [44]:
pdb_to_ligandpdb(pdb_id='5S6Y',ligand_name = 'WUJ')

401 


In [27]:
pdb_to_pdbnew(pdb_id='5S6Y')

In [51]:
with open('./error_list/cavity_empty_list.txt', 'r') as f:
    text = f.read()
pdb_ids = text.split(',')

In [47]:
# (删除mol2文件夹下的非pdb文件，慎用)
# for i in tqdm(pdb_ids):
#     try:
#         pdb_id_path=os.listdir(f'./gett_pdb/{i}/')
#         for j in pdb_id_path:
#             if j[-4:] == 'mol2' or j[-4:]=='.sdf':
#                 continue
#             else:
#                 shutil.rmtree(f'./gett_pdb/{i}/{j}')
#         pdb_id_path2=os.listdir(f'./gett_pdb/{i}/{i}_mol2')
#         for j in pdb_id_path2:
#             if j ==f'{i}.pdb':
#                 continue
#             elif j[-4:] == '.pdb' or j[-5:]=='.mol2':
#                  os.remove(f'./gett_pdb/{i}/{i}_mol2/{j}')
#             else:
#                 try:
#                     shutil.rmtree(f'./gett_pdb/{i}/{i}_mol2/{j}')
#                 except:
#                     pass
#     except:
#         pass

100%|██████████| 1510/1510 [00:00<00:00, 8391.89it/s]


In [53]:
for i in tqdm(pdb_ids):
    try:
        pdb_to_pdbnew(pdb_id=i)
        os_listdir = os.listdir(f'./gett_pdb/{i}/')
        for j in os_listdir:
            if j[-4:] == '.sdf':
                ligand_name=j[:-10]
                pdb_to_ligandpdb(pdb_id=i,ligand_name = ligand_name)
    except:
        continue


100%|██████████| 722/722 [00:02<00:00, 327.06it/s]


In [50]:
for i in tqdm(pdb_ids):
    path = f'./gett_pdb/{i}/{i}_mol2/ligand/'
    a = []
    b={}
    try:
        for j in os.listdir(path):
            a.append(os.path.getsize(f'./gett_pdb/{i}/{i}_mol2/ligand/{j}'))
            b[j]=os.path.getsize(f'./gett_pdb/{i}/{i}_mol2/ligand/{j}')
        c = max(a)
        for k in b:
            if b[k] == c:
                continue
            else:
                os.remove(f'./gett_pdb/{i}/{i}_mol2/ligand/{k}')
    except:
        continue
    # 保留配体最大的那个

100%|██████████| 1510/1510 [00:00<00:00, 142534.97it/s]
